In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import ImageTk, Image
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf

2024-07-02 21:55:11.758494: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = load_model('model2.h5') # model extension changed to .h5 due to conflicts with .keras

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [4]:
def process_image(image_path):
    img = imread(image_path)
    img = resize(img, (128,128,3))
    img_array = np.array(img) / 255
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [5]:
def predict_tumor(img_array):
  y_predict = model.predict(img_array)
  return y_predict, np.argmax(y_predict)

In [6]:
import math

In [7]:
def get_confidence(y_predict, y_class):
  return math.ceil(y_predict[0][y_class]*100)

In [8]:
class ImageApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classification App")
        self.root.geometry("500x500")

        self.label = tk.Label(root, text="Upload an image to classify")
        self.label.pack(pady=20)

        self.upload_button = tk.Button(root, text="Upload Image", command=self.upload_image)
        self.upload_button.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=20)

        self.result_label = tk.Label(root, text="", font=("Helvetica", 16))
        self.result_label.pack(pady=20)

    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.display_image(file_path)
            img_array = process_image(file_path)
            y_predict, y_class = predict_tumor(img_array)
            confidence = get_confidence(y_predict, y_class)
            self.display_result(categories[y_class], confidence)

    def display_image(self, file_path):
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)
        self.image_label.configure(image=img)
        self.image_label.image = img

    def display_result(self, prediction, confidence):
        self.result_label.config(text=f"Prediction: {prediction} with {confidence}% confidence")

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageApp(root)
    root.mainloop()

2024-07-02 21:59:33.128 python[84493:5304855] +[CATransaction synchronize] called within transaction
2024-07-02 21:59:33.294 python[84493:5304855] +[CATransaction synchronize] called within transaction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
